# Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#import twint

I decided to use Twint to scrape data from Twitter. I hesitated with Tweepy but this library is limited with the API limitations... As I want to collect all Twitter mentions from September, that library might not be well-suited.

# Twitter data collection

In [ ]:
#configuration
#config = twint.Config()
#config.Search = "PS5"
#config.Lang = "en"
#config.Limit = 5
#config.Since = '2020–09–01 00:01:00'
#config.Until = "2020–9–28 00:00:00"
#config.Store_csv = True
#config.Output = "test.csv"
#running search
#twint.run.Search(config)

In [2]:
import twitter_scraper

In [3]:
from twitter_scraper import get_tweets

In [4]:
tweets = get_tweets("PS5",pages=5)

In [5]:
tweets

<generator object get_tweets at 0x10cf59550>

In [6]:
tweets_df = pd.DataFrame()

In [7]:
for tweet in tweets:
    print('Keys:', list(tweet.keys()), '\n')
    break

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [8]:
for tweet in tweets:
    _ = pd.DataFrame({'text' : [tweet['text']],
                    'isRetweet' : tweet['isRetweet'],
                    'replies' : tweet['replies'],
                    'retweets' : tweet['retweets'],
                    'likes' : tweet['likes']
                    })
    tweets_df = tweets_df.append(_, ignore_index = True)
tweets_df.head()

""


In [9]:
for i in tweets:
    tweets_df.append(i)

In [10]:
tweets_df

""


In [11]:
import twitterscraper
import datetime as dt

In [12]:
from twitterscraper import query_tweets

In [13]:
tweets = query_tweets('bitcoin', limit=10, begindate=dt.date.today(), enddate=dt.date.today(), lang='en')

In [15]:
for i in tweets:
    print(i)

In [16]:
df = pd.DataFrame(t.__dict__ for t in tweets)

In [17]:
df

""


In [19]:
pip install GetOldTweets3

Note: you may need to restart the kernel to use updated packages.


In [20]:
import GetOldTweets3 as got

In [21]:
tweetCriteria = got.manager.TweetCriteria().setQuerySearch('PS5')\
                                           .setSince("2020-09-01")\
                                           .setUntil("2020-09-30")\
                                           .setMaxTweets(10)
tweet = got.manager.TweetManager.getTweets(tweetCriteria)[0]
print(tweet.text)

An error occured during an HTTP request: HTTP Error 404: Not Found
Try to open in browser: https://twitter.com/search?q=PS5%20since%3A2020-09-01%20until%3A2020-09-30&src=typd


SystemExit: 

/Users/salim/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [22]:
import twint
c = twint.Config()
c.Search = "ps5"
c.Store_object = True
c.Limit = 100
twint.run.Search(c)
tlist = c.search_tweet_list

RuntimeError: This event loop is already running

CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 1.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 8.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 27.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 64.0 secs


In [23]:
c=twint.Config()
c.Hide_output=False #but it ignores this
c.Store_object=True
c.Store_csv=True
c.Since="2020-09-01"
c.User_full=True
c.Limit=100 #I have tested up to 1000 ok
c.Output="testing.csv" #but it ignores this

srchname="(from:RealDonaldTrump)"
srch=srchname+" max_id:"+str(1307047063197224961)

twint.run.Search(c)
tlist = c.search_tweet_list
print(tlist)

RuntimeError: This event loop is already running

CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 1.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 125.0 secs
